## New Summarizer experiments1

What I need to do

- Retrieve the today news, article from the scrapper database.

- Summarize them with good prompt.

- display them on a UI.

In [1]:
%load_ext dotenv

In [2]:
%dotenv ./.env_prod -o

In [3]:
from src.shared.database import execute_query, generate_database_connection

In [4]:
yesterday_article_query = "select content, title, posted_at,url from article where posted_at::date = CURRENT_DATE - interval '1 day'"

In [5]:
from os import getenv

In [6]:
database_user = getenv('POSTGRES_USER')
database_password = getenv('POSTGRES_PASSWORD')
database_host = getenv('POSTGRES_HOST')
database_port = getenv('POSTGRES_PORT')
database_name = getenv('POSTGRES_DB')

In [7]:
database_credentials = {
    'user': database_user,
    'password': database_password,
    'host': database_host,
    'port': database_port,
    'database': database_name
}

In [8]:
connection = generate_database_connection(database_crendentials=database_credentials)

In [9]:
connection

<connection object at 0x10af84040; dsn: 'user=esp_py password=xxx dbname=congo_news_db host=107.173.6.77 port=5434', closed: 0>

In [10]:
results =execute_query(query=yesterday_article_query, database_connection=connection)

In [11]:
results[0].url

'https://econewsrdc.com/presence-remarquee-de-lanapi-a-la-14eme-edition-du-forum-europafrica-investment-a-bruxelles/'

In [12]:
import pandas as pd

In [13]:
results_df = pd.DataFrame.from_records(results)

In [14]:
results_df.columns = ["content", "title", "posted_at", "url"]

In [382]:
from pathlib import Path

In [383]:
current_directory = Path.cwd().joinpath("datasets", "today_news")

In [386]:
current_directory.mkdir(exist_ok=True)

In [388]:
from datetime import datetime

In [389]:
today = datetime.now().strftime("%Y-%m-%d")

In [390]:
results_df.to_csv(current_directory.joinpath(f"{today}-news.csv"))

In [15]:
results_df.content

0     Dans le cadre de la mise en œuvre de son Plan ...
1     Les membres du gouvernement provincial de Mai-...
2     VODACASH S.A, filiale de Vodacom Congo (RDC) S...
3     octobre 10, 2024 Kinshasa- Financement des tra...
4     Les Leopards de la RDC signent une précieuse v...
                            ...                        
67    ‘ »Kabila-Lumumba au grade de Grand Officier, ...
68    Sur instruction du Président de la République,...
69    L’ultimatum de 72h lancé le 07 octobre 2024 pa...
70    Moins tranchants dans le jeu, les fauves congo...
71    Au moins 47.911 personnes sont attendues pour ...
Name: content, Length: 72, dtype: object

In [16]:
results_df = results_df.drop_duplicates(subset="content")

In [17]:
results_df.head()

,content,title,posted_at,url
0,Dans le cadre de la mise en œuvre de son Plan ...,Présence remarquée de l’ANAPI à la 14ème éditi...,2024-10-10 00:00:00,https://econewsrdc.com/presence-remarquee-de-l...
1,Les membres du gouvernement provincial de Mai-...,Mai-Ndombe : le gouvernement provincial publié...,2024-10-10 08:59:00,https://actualite.cd/2024/10/10/mai-ndombe-le-...
2,"VODACASH S.A, filiale de Vodacom Congo (RDC) S...",Partenariat stratégique entre VODACASH et la S...,2024-10-10 00:00:00,https://econewsrdc.com/partenariat-strategique...
3,"octobre 10, 2024 Kinshasa- Financement des tra...",Kinshasa- Financement des travaux du projet 3 ...,2024-10-10 00:00:00,https://depeche.cd/2024/10/10/kinshasa-finance...
4,Les Leopards de la RDC signent une précieuse v...,"Elim/CAN 2025 : Toujours aussi poussifs, les L...",2024-10-10 19:18:00,https://actualite.cd/index.php/2024/10/10/elim...


In [18]:
from src.rag.components.generator import LLamaCppGeneratorComponent

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [20]:
prompt = "You are a french news reporter"

In [21]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [22]:
llama_cpp_generator._ping_api()

True

prompting doesn't seem to wokr, let us use clustering and prompting.

The step will be computing the embedding of the documents, and then cluster the document. Then for each cluster of documents, we will generate a summary.

embedding_model_id = "dunzhang/stella_en_400M_v5"

In [23]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [24]:
from pathlib import Path

In [25]:
model_path = Path.cwd().joinpath("models")

In [26]:
embedding_model_path = model_path.joinpath(embedding_model_id)

In [27]:

transformer_kwargs = {"model_name_or_path": embedding_model_path.__str__(),
                      "trust_remote_code": True,
                      "device": "cpu",
                      "config_kwargs": {"use_memory_efficient_attention": False,
                                        "unpad_inputs": False},
                      "cache_folder": model_path}

In [28]:
from sentence_transformers import SentenceTransformer

In [29]:
sentence_transformer_model = SentenceTransformer(
    **transformer_kwargs)

Some weights of the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
today_news_embeddings = sentence_transformer_model.encode(
    results_df.content.tolist())

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:1141: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Not just clustering but let start with paragraph spliting and then embedding.

In [31]:
today_news_embeddings

array([[-0.2382813 ,  0.0202584 , -0.96147484, ...,  0.14334454,
         0.00407858, -0.4053433 ],
       [-0.08295427,  0.01973198, -1.2731227 , ...,  0.47395864,
         0.06307081,  0.792102  ],
       [-0.20636846, -0.4537534 , -1.5298388 , ..., -0.30492252,
        -0.07919986, -0.33250016],
       ...,
       [-0.21599424,  0.3710519 , -1.4436466 , ..., -0.22783493,
         0.09507482, -0.10069045],
       [-0.13976383, -0.17892939, -1.5407742 , ...,  0.7428794 ,
         0.15088311, -0.26181147],
       [-0.39839163, -0.24835615, -1.5162802 , ..., -0.04720241,
        -0.14301531, -0.60298836]], dtype=float32)

In [32]:
today_news_embeddings.shape

(72, 1024)

In [33]:
from sklearn.cluster import KMeans

In [34]:
import matplotlib.pyplot as plt

In [35]:
axes = plt.figure(figsize=(5, 10))

<Figure size 500x1000 with 0 Axes>

In [36]:
similarities = sentence_transformer_model.similarity(today_news_embeddings, today_news_embeddings)

In [37]:
import seaborn as sb

In [38]:
#plt.figure(figsize=(50, 50))
#sb.heatmap(similarities, cmap="Blues", annot=True)

In [39]:
results_df.iloc[25]

content      À l’occasion, l’autorité provinciale et les au...
title        Maniema : la campagne de vaccination contre la...
posted_at                                  2024-10-10 05:38:59
url          https://actu30.cd/2024/10/maniema-la-campagne-...
Name: 25, dtype: object

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score

In [41]:
def find_best_estimator (X):
    K_mean_estimators = [
        (f"KMeans_{i}", KMeans(n_clusters=i, random_state=42)) for i in [ 5, 7, 10, 12]]

    best_estimator = None
    best_metric = float("-inf")
    for estimator_name, estimator in K_mean_estimators :
        estimator.fit(X)
        labels = estimator.labels_
        score = silhouette_score(
            today_news_embeddings, labels, metric="cosine")
        if score > best_metric :
            best_metric = score
            best_estimator = estimator
        print(estimator_name, score)
    return best_estimator

In [ ]:
A value of 1,

In [42]:
best_estimator = find_best_estimator(today_news_embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KMeans_5 0.08535227
KMeans_7 0.1300366
KMeans_10 0.106102824
KMeans_12 0.14421302


In [43]:
best_estimator

KMeans(n_clusters=12, random_state=42)

In [44]:
results_df["labels"] = best_estimator.labels_

In [45]:
results_df.query("labels ==11")

,content,title,posted_at,url,labels
0,Dans le cadre de la mise en œuvre de son Plan ...,Présence remarquée de l’ANAPI à la 14ème éditi...,2024-10-10 00:00:00,https://econewsrdc.com/presence-remarquee-de-l...,11
6,"octobre 10, 2024 Allemagne : “Le Président Tsh...",Allemagne : “Le Président Tshisekedi a déjà fa...,2024-10-10 00:00:00,https://depeche.cd/2024/10/10/allemagne-le-pre...,11
23,"Cet événement, organisé par Louise Nzanga Rama...",RDC : Judith Suminwa renforce les liens avec l...,2024-10-10 11:56:44,https://actu30.cd/2024/10/rdc-judith-suminwa-r...,11
38,En séjour en Allemagne dans le cadre de la con...,Allemagne : Judith Suminwa échange avec la dia...,2024-10-10 00:00:00,https://www.mediacongo.net/article-actualite-1...,11
55,La coopération entre la République Démocratiqu...,Processus de Luanda et de Nairobi : le soutien...,2024-10-10 00:00:00,https://www.mediacongo.net/article-actualite-1...,11
64,"D’après l’assemblée nationale congolaise, les ...",RDC – France : Vital Kamerhe et son homologue ...,2024-10-10 16:41:27,https://actu30.cd/2024/10/rdc-france-vital-kam...,11


In [46]:
summarization_prompt = """
Give a summary in french of the following document:
{{content}}
 Describes it in a style of a french new paper reporters.

Don't summarize each document separately, the content in all the documents should be summarized.

The summary should be in french not in English
"""

In [56]:
first_cluster = results_df.query("labels == 5")
first_cluster

,content,title,posted_at,url,labels
11,L'inhumation de 11 corps identifiés des habita...,Naufrage sur le lac Kivu: inhumation ce jeudi ...,2024-10-10 16:39:00,https://actualite.cd/2024/10/10/naufrage-sur-l...,5
37,L'Administrateur délégué de l'hôpital Biamba M...,Kinshasa : l'AD de l'hôpital Biamba Marie Muto...,2024-10-10 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
43,Une veillée mortuaire est prévue vendredi 10 o...,Kinshasa : une veillée mortuaire vendredi au P...,2024-10-10 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
45,Alors que l'état de santé du plus ancien Chef ...,Le Gouvernement camerounais dément la rumeur s...,2024-10-10 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
46,À 48 heures de la date marquant le décès de pa...,"Mémorandum des culturels : "" Nous réitérons no...",2024-10-10 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
48,Rafael Nadal a annoncé ce jeudi 10 octobre 202...,"""Je quitte le tennis professionnel"", annonce l...",2024-10-10 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
56,Considérant les mérites exceptionnels de l’int...,RDC: Dikembe Mutombo admis à titre posthume da...,2024-10-10 20:27:00,https://actualite.cd/2024/10/10/rdc-dikembe-mu...,5
67,"‘ »Kabila-Lumumba au grade de Grand Officier, ...",RDC : Mutombo Dikembe élevé au rang de grand o...,2024-10-10 16:36:37,https://actu30.cd/2024/10/rdc-mutombo-dikembe-...,5


In [48]:
first_cluster.content.str.cat(sep="\n")

'octobre 10, 2024 Kinshasa- Financement des travaux du projet 3 communes: Lobbying payant du sénateur Gecoco Mulumba Par :    La rédaction \n \n \n Kinshasa, 10 Octobre 2024- Le plaidoyer du sénateur Gecoco Mulumba auprès du ministre des finances pour décanter la situation de paiement des entreprises chargées  de la construction des routes dans les communes de Gombe, Kasa-vubu et Limete a payé. “Ce mercredi 09 octobre 2024, j’ai été remercié le Ministre National des Finances pour avoir accédé à mon plaidoyer au sujet du paiement de fonds aux entreprises concernées dans le Projet 3 Communes (Gombe, Kasa-Vubu et Limete). Il y a 10 jours, en effet, je l’ai rencontré à ce sujet et il a promis de décanter la situation. Je tiens à remercier Monsieur le Président de la République, initiateur du ce projet qui permettra une mobilité aisée de notre population”,  a-t-il fait savoir. D’après l’ancien vice gouverneur de Kinshasa, ces entreprises vont devoir dans 3 ou 4 jours, toucher un peu plus de

In [49]:
llama_cpp_generator.generate_chat_input(
    template_values={"content": first_cluster.content.str.cat(sep="\n")}, prompt_template=summarization_prompt)

[{'role': 'system', 'content': 'You are a french news reporter'},
 {'role': 'user',
  'content': '\nGive a summary in french of the following document:\noctobre 10, 2024 Kinshasa- Financement des travaux du projet 3 communes: Lobbying payant du sénateur Gecoco Mulumba Par :    La rédaction \n \n \n Kinshasa, 10 Octobre 2024- Le plaidoyer du sénateur Gecoco Mulumba auprès du ministre des finances pour décanter la situation de paiement des entreprises chargées  de la construction des routes dans les communes de Gombe, Kasa-vubu et Limete a payé. “Ce mercredi 09 octobre 2024, j’ai été remercié le Ministre National des Finances pour avoir accédé à mon plaidoyer au sujet du paiement de fonds aux entreprises concernées dans le Projet 3 Communes (Gombe, Kasa-Vubu et Limete). Il y a 10 jours, en effet, je l’ai rencontré à ce sujet et il a promis de décanter la situation. Je tiens à remercier Monsieur le Président de la République, initiateur du ce projet qui permettra une mobilité aisée de not

In [50]:
summaries = []
for id, news_group in results_df.groupby("labels"):
    news = news_group.content.str.cat(sep="\n")
    summary = llama_cpp_generator.run(template_values={"content": news}, prompt_template=summarization_prompt)
    summaries.append(summary)
    print(summary)
    print("---" * 10)

La quatrième phase de la campagne de vaccination contre la poliomyélite a été lancée mercredi 9 octobre à Inongo, dans la province de Maniema. Au total, 457.201 enfants de zéro à cinquante-neuf mois seront vaccinés du 10 au 12 octobre. Le lancement a été fait par le gouverneur de province, Nkoso Kevani Lebon, dans la commune de Mpolo Maurice. Les équipes de vaccination passeront de porte à porte pour atteindre les enfants ciblés. L'autorité provinciale et les autorités sanitaires ont lancé plusieurs messages de sensibilisation aux parents pour faire vacciner leurs enfants. La campagne, qui vise à vacciner plus d'un million d'enfants dans les 16 zones de santé de Maniema, va s'étendre sur trois jours. Le vice-gouverneur Célestin Kayembe a présidé la cérémonie officielle du lancement. La province du Kwilu continue d'enregistrer les cas de Mpox, dont 220 depuis la déclaration de l'épidémie en 2024, avec un décès. Le ministre provincial de la santé, Arnold Kapalati, précise que certains ca

I will have to stop here, and then start writing about the learning. 

- Another apporach will be collect the document from radiookapi and fine tune the model.
-   Deploy the solution with streamlit

### Trying Topic Modeling and Then Summarizing

In  this section we will use topic modelling and then summarize the documents in each topic.

In [57]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

In [65]:
from spacy.lang.fr.stop_words import STOP_WORDS as french_stop_words

In [108]:
CONGO_WORDS = {
    'congo',
    'congolais',
    "congolaise"
    'rdc',
    'drc',
    '-',
    'https',
    'rdcongo',
    'drc',
    'drcongo'}

In [109]:
words_to_remove = CONGO_WORDS.union(french_stop_words)

In [110]:
hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean',
                        cluster_selection_method='eom', prediction_data=True)

In [111]:
umap_model = UMAP(n_neighbors=15, n_components=15,
                  min_dist=0.0, metric='cosine')

In [112]:
count_vectorizer = CountVectorizer(
    stop_words=list(words_to_remove), ngram_range=(1, 2))

In [113]:
ctfidf_model = ClassTfidfTransformer()

In [114]:
representation_model = KeyBERTInspired()

In [115]:
topic_model = BERTopic(
    embedding_model=sentence_transformer_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=best_estimator,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=count_vectorizer,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    # Step 6 - (Optional) Fine-tune topic representations
    representation_model=representation_model
)

In [116]:
topics, probs = topic_model.fit_transform(results_df.content)

In [118]:
probs

In [117]:
topic_model.visualize_topics()

In [126]:
new_topics  = topic_model.reduce_topics(
    results_df.content, nr_topics=7)


In [127]:
new_topics.visualize_topics()

In [128]:
topic_model.visualize_documents(results_df.content, embeddings=reduced_embebddings)

In [96]:
find_best_estimator(reduced_embeddings)

KMeans_5 0.11653532
KMeans_7 0.103666335
KMeans_10 0.12566169
KMeans_12 0.16120532


KMeans(n_clusters=12, random_state=42)

The bert topic modeling is not really topic model, it is mostly document reprenstaiton plus document clustering. So if we finetune our clustering algorithm better we can get better results.

### Analyzing inter cluster Similarities.

In [129]:
results_df.head()

,content,title,posted_at,url,labels
0,Dans le cadre de la mise en œuvre de son Plan ...,Présence remarquée de l’ANAPI à la 14ème éditi...,2024-10-10 00:00:00,https://econewsrdc.com/presence-remarquee-de-l...,11
1,Les membres du gouvernement provincial de Mai-...,Mai-Ndombe : le gouvernement provincial publié...,2024-10-10 08:59:00,https://actualite.cd/2024/10/10/mai-ndombe-le-...,7
2,"VODACASH S.A, filiale de Vodacom Congo (RDC) S...",Partenariat stratégique entre VODACASH et la S...,2024-10-10 00:00:00,https://econewsrdc.com/partenariat-strategique...,3
3,"octobre 10, 2024 Kinshasa- Financement des tra...",Kinshasa- Financement des travaux du projet 3 ...,2024-10-10 00:00:00,https://depeche.cd/2024/10/10/kinshasa-finance...,1
4,Les Leopards de la RDC signent une précieuse v...,"Elim/CAN 2025 : Toujours aussi poussifs, les L...",2024-10-10 19:18:00,https://actualite.cd/index.php/2024/10/10/elim...,10


In [134]:
document_index = results_df.query("labels == 1").index

In [135]:
vectors = today_news_embeddings[document_index]

In [138]:
sentence_transformer_model.similarity(vectors ,  vectors)

tensor([[1.0000, 0.6475, 0.6588, 0.6429, 0.6914, 0.5992],
        [0.6475, 1.0000, 0.8501, 0.9083, 0.6733, 0.6499],
        [0.6588, 0.8501, 1.0000, 0.9174, 0.6630, 0.7389],
        [0.6429, 0.9083, 0.9174, 1.0000, 0.6406, 0.6991],
        [0.6914, 0.6733, 0.6630, 0.6406, 1.0000, 0.7347],
        [0.5992, 0.6499, 0.7389, 0.6991, 0.7347, 1.0000]])

In [374]:
def analyse_embeddings(dataframe, embeddings, index, label_column="labels"):
    """ take an matrix of embeddings, and the labels.
    for each label compute the cosine similarity of document with that label.
    """
    document_in_index = dataframe.query(f"{label_column} == {index}")
    with pd.option_context('display.max_colwidth', None):
        display(document_in_index.title)
    document_index = document_in_index.index
    vectors = embeddings[document_index]
    return sentence_transformer_model.similarity(vectors ,  vectors)

    

In [167]:
similarity_matrix = analyse_embeddings(results_df, today_news_embeddings, 4)

14                                         Tennis: La FRDCT participe à l’Assemblée Générale de l’ITF à Hong Kong avec son président, l’ingénieur Georges Koshi Gimeya | Actualite.cd
18                                                                           Francophonie : après le Sommet de Paris, que devrait faire la RDC ? (Par Mabiala Ma-Umba) | Actualite.cd
20                                    Conseil des droits de l’homme de l’ONU : Chantal Chambu salue le leadership du chef de l’État ayant conduit à l’élection de la RDC comme membre
29                                                                         Malgré son élection, « la RDC ne mérite pas de siéger au Conseil des droits de l'homme de l'ONU » (Diongo)
31                               Élection de la RDC au Conseil des droits de l'homme de l'ONU : "lorsque le pays avance, il faut que tout le monde puisse souscrire" (Patrick Muyaya)
60    Élection de la RDC au Conseil des droits de l’Homme : la reconnaissance de la marche

In [169]:
similarity_matrix

tensor([[1.0000, 0.6300, 0.5833, 0.5917, 0.6243, 0.6672, 0.6267],
        [0.6300, 1.0000, 0.6451, 0.7164, 0.7002, 0.6536, 0.8410],
        [0.5833, 0.6451, 1.0000, 0.7521, 0.8569, 0.8247, 0.6973],
        [0.5917, 0.7164, 0.7521, 1.0000, 0.8109, 0.7743, 0.7533],
        [0.6243, 0.7002, 0.8569, 0.8109, 1.0000, 0.8897, 0.7262],
        [0.6672, 0.6536, 0.8247, 0.7743, 0.8897, 1.0000, 0.6680],
        [0.6267, 0.8410, 0.6973, 0.7533, 0.7262, 0.6680, 1.0000]])

In [170]:
from sklearn.cluster import DBSCAN

In [179]:
db_scan_model.fit(today_news_embeddings)

DBSCAN(eps=0.75, metric='cosine', min_samples=1)

In [180]:
db_scan_model.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [181]:
from sklearn.metrics import pairwise_distances

In [202]:
sentence_transformer_model.similarity(today_news_embeddings[0], today_news_embeddings[0])[0].item()

1.0

In [203]:
def transformer_cosine_similarity(X, Y):
    return sentence_transformer_model.similarity(X, Y).item()

In [366]:
db_scan_model = DBSCAN(eps=0.3, min_samples=5, metric="cosine", algorithm="auto")

In [367]:
db_scan_model.fit(today_news_embeddings)

DBSCAN(eps=0.3, metric='cosine')

In [368]:
db_scan_model.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  2, -1, -1, -1, -1, -1, -1,
       -1,  1, -1,  1, -1, -1, -1,  0,  2,  2, -1, -1,  1, -1,  1,  0, -1,
       -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  2, -1, -1,  2, -1,
        0, -1, -1,  2])

In [369]:
results_df["labels_db_scan"] = db_scan_model.labels_

In [297]:
### Ned to explore nearest neibords models.

Need to explore why db scan is not working, and then work out the nearest neibor stuff

 Another approach to try to get the best document actually should be trying to cluster document based on their titles.

In [220]:
from sklearn.neighbors import NearestNeighbors

In [335]:
neighbors_model = NearestNeighbors(
    radius=0,
    algorithm="auto",
    leaf_size=30,
    metric="cosine",
    metric_params={},
)

In [336]:
neighbors_model.fit(today_news_embeddings)

NearestNeighbors(metric='cosine', metric_params={}, radius=0)

In [337]:
today_news_embeddings[0].reshape(1, -1)

array([[-0.2382813 ,  0.0202584 , -0.96147484, ...,  0.14334454,
         0.00407858, -0.4053433 ]], dtype=float32)

In [338]:
distances, indices = neighbors_model.kneighbors(today_news_embeddings[10].reshape(1, -1))

In [349]:
neighbors_distances, neighbor_indices = neighbors_model.radius_neighbors(today_news_embeddings, radius=0.3, return_distance=True)

In [350]:
neighbor_indices[10]

array([10, 25, 26, 71])

In [348]:
neighbors_distances[10]

array([0.        , 0.24850214, 0.20240474, 0.24215406], dtype=float32)

In [316]:
indices[0]

array([10, 26, 71, 25, 66])

In [270]:
results_df.query("labels == 0")

,content,title,posted_at,url,labels
10,La quatrième phase de la campagne de vaccinati...,Mai-Ndombe : près de 500 000 enfants attendus ...,2024-10-10 12:07:00,https://actualite.cd/index.php/2024/10/10/mai-...,0
25,"À l’occasion, l’autorité provinciale et les au...",Maniema : la campagne de vaccination contre la...,2024-10-10 05:38:59,https://actu30.cd/2024/10/maniema-la-campagne-...,0
26,La cérémonie du lancement officiel de cette ca...,Lomami : Début de la campagne de vaccination c...,2024-10-10 05:45:26,https://actu30.cd/2024/10/lomami-debut-de-la-c...,0
63,La province du Kwilu continue d’enregistrer le...,Mpox au Kwilu : 9 morts sur plus de 200 cas en...,2024-10-10 06:44:00,https://actualite.cd/2024/10/10/mpox-au-kwilu-...,0
71,Au moins 47.911 personnes sont attendues pour ...,Plus de 47 000 personnes visées par la vaccina...,2024-10-10 08:41:00,https://actualite.cd/2024/10/10/plus-de-47-000...,0


In [381]:
analyse_embeddings(results_df, today_news_embeddings, -1, "labels_db_scan")

0                                                                                                                                                         Présence remarquée de l’ANAPI à la 14ème édition du Forum «EuropAfrica Investment» à Bruxelles
1                                                                                                                                                            Mai-Ndombe : le gouvernement provincial publié, aucune femme nommée ministre | Actualite.cd
2                                                                                                                                             Partenariat stratégique entre VODACASH et la SFI pour la promotion des services financiers mobiles en RDC 
3                                                                                                                                                     Kinshasa- Financement des travaux du projet 3 communes: Lobbying payant du sénateur Gecoco Mulumba
4   

tensor([[1.0000, 0.4667, 0.5781,  ..., 0.4595, 0.5536, 0.4807],
        [0.4667, 1.0000, 0.4042,  ..., 0.4468, 0.4951, 0.4223],
        [0.5781, 0.4042, 1.0000,  ..., 0.3881, 0.4901, 0.4765],
        ...,
        [0.4595, 0.4468, 0.3881,  ..., 1.0000, 0.4108, 0.4690],
        [0.5536, 0.4951, 0.4901,  ..., 0.4108, 1.0000, 0.4076],
        [0.4807, 0.4223, 0.4765,  ..., 0.4690, 0.4076, 1.0000]])

In [377]:
today_news_embeddings[indices[0]].shape

(5, 1024)

In [277]:
similarities = sentence_transformer_model.similarity(
    today_news_embeddings[indices[0]], today_news_embeddings[indices[0]])

In [278]:
1 - similarities

tensor([[-3.5763e-07,  2.0240e-01,  2.4215e-01,  2.4850e-01,  3.6869e-01],
        [ 2.0240e-01, -2.3842e-07,  2.5243e-01,  3.1303e-01,  4.1540e-01],
        [ 2.4215e-01,  2.5243e-01,  4.7684e-07,  2.8232e-01,  2.6365e-01],
        [ 2.4850e-01,  3.1303e-01,  2.8232e-01, -1.1921e-07,  4.0719e-01],
        [ 3.6869e-01,  4.1540e-01,  2.6365e-01,  4.0719e-01, -9.5367e-07]])

Kmean is not giving good results because some cluster are not correct., Let try with the title embeddings.